# Lad Accounting dashboard

In [ ]:
# Installing required packages
# DESATUALIZADO 
!pip install dash-bootstrap-components
!pip install pandas
!pip install xlrd
!pip install openpyxl

In [6]:
from typing import Container
from _plotly_utils.importers import relative_import

from jupyter_dash import JupyterDash
from dash.html.Figure import Figure
from dash_bootstrap_components._components.Card import Card
import dash_table
from dash.html.H3 import H3
from dash.html.P import P
from dash.html.Span import Span
from dash_bootstrap_components._components import CardHeader
from dash_bootstrap_components._components.Col import Col
from dash_bootstrap_components._components.Row import Row

import dash_core_components as dcc
import dash_html_components as html
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output
from pandas.io.formats import style

import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import pandas as pd
import numpy as np
import os

app = JupyterDash(external_stylesheets=[dbc.themes.BOOTSTRAP])

pio.templates.default = "plotly_dark"


# ----------------------------------------------  Leitura  ---------------------------------------------- #


df_relatorio_geral = pd.DataFrame()

for dataframe in os.listdir('relatorios/2021'):
    df_relatorio_geral = df_relatorio_geral.append(pd.read_excel(os.path.join('relatorios/2021', dataframe)))


# ----------------------------------------------  Layout  ---------------------------------------------- #


app.layout = html.Div([

    # ------------ Nav ----------------------------------------------- #

    html.Div(
        dbc.Row([
            html.Img(id="logo", src=app.get_asset_url("LAD.png"), height=70),
            html.H2("LAD Accounting Dashboard", style={'margin-top': '5px', 'color':'white'}),
            ],
            style={'margin': 'auto', 'justify-content':'center'}
        ),
        style={'padding': '2rem', 'background-color': '#212529'}
    ),

    # ------------ Seção 1 - seleção --------------------------------- #
    
    dbc.Col([
        dbc.Row([
            dbc.Col(
                html.H5('2021'),
                width=2,
                style={'color':'white', 'text-align':'center', 'margin': 'auto'}
            ),

            dbc.Col(
                dcc.Slider(
                    id='month_slider',
                    min=df_relatorio_geral['Mês'].min(),
                    max=df_relatorio_geral['Mês'].max(),
                    value=df_relatorio_geral['Mês'].max(),
                    marks={
                            1: {'label': 'Janeiro'},
                            2: {'label': 'Fevereiro'},
                            3: {'label': 'Março'},
                            4: {'label': 'Abril'},
                            5: {'label': 'Maio'},
                            6: {'label': 'Junho'},
                            7: {'label': 'Julho'},
                            8: {'label': 'Agosto'},
                            9: {'label': 'Setembro'},
                            10: {'label': 'Outubro'},
                            11: {'label': 'Novembro'},
                            12: {'label': 'Dezembro'}
                    },
                    included=False
                )
            )],
            style={'margin': '0', 'background-color': 'rgb(17, 17, 17)', 'padding': '1.25rem'}
        )],
    style={'margin': '1rem 0'}
    ),

    # ------------ Seção 2 - Cards ----------------------------------- #

    dbc.Col(
        dbc.Row([

            dbc.Col(
                dbc.Card([
                    dbc.CardHeader('Storage', style={'border-bottom': 'none'}),
                    dbc.CardBody([
                        dbc.Col(
                            dcc.Graph(
                                id='fig_storage_card'
                            )
                        ),
                        dbc.Row([
                            dbc.Col([
                                html.Span('Utilizado'),
                                html.H5(id='total_utilizado_storage', style={'color': 'white'}),
                            ]),
                            dbc.Col([
                                html.Span('Disponível'),
                                html.H5(id='total_disponivel_storage', style={'color': 'white'})
                            ]),
                            dbc.Col([
                                html.Span('Capacidade'),
                                html.H5('134206 GB', style={'color': 'white'})
                            ])
                        ])
                    ], style={'padding-top':'0'}
                    )
                    ],
                    className='shadow text-center', 
                    style={'border': 'none','background-color': 'rgb(17, 17, 17)', 'color': 'white'}
                ), width=6
            ),

            dbc.Col(
                dbc.Card([
                    dbc.CardHeader('Horas núcleo total', style={'border-bottom': 'none'}),
                    dbc.CardBody([
                        dbc.Col(
                            dcc.Graph(
                                id='graph_horas_nucleo_total'
                            )
                        ),
                        dbc.Row([
                            dbc.Col([
                                html.Span('Utilizado'),
                                html.H5(id='total_utilizado_horas_nucleo', style={'color': 'white'}),
                            ]),
                            dbc.Col([
                                html.Span('Disponível'),
                                html.H5(id='total_disponivel_horas_nucleo', style={'color': 'white'})
                            ]),
                            dbc.Col([
                                html.Span('Capacidade'),
                                html.H5(id='capacidade_horas_nucleo', style={'color': 'white'})
                            ])
                        ])
                    ], style={'padding-top':'0'}
                    )

                    ],
                    className='shadow text-center', 
                    style={'border': 'none', 'background-color': 'rgb(17, 17, 17)', 'color': 'white'}
                ), width=6
            )
            
        ])
    ),


    # ------------ Seção 3 - Máquina em cluster - 24x7 ----------------------- #

    dbc.Col(
        dbc.Row([

            dbc.Col([
                dbc.Card([
                    dbc.CardHeader('Utilização de Máquina em 24x7 por grupo', style={'border-bottom': 'none', 'background-color': 'rgb(20, 200, 255)', 'color': 'white'}),
                    dcc.Graph(
                        id='graph_maquina24x7',
                    )],
                    className='shadow',
                    style={'border': 'none'}
                )],
                width=6
            ),

            dbc.Col([
                dbc.Card([
                    dbc.CardHeader('Utilização de Máquina em Cluster por grupo', style={'border-bottom': 'none', 'background-color': 'rgb(30, 110, 255)', 'color': 'white'}),
                    dcc.Graph(
                        id='graph_maquina_cluster',
                    )],
                    className='shadow',
                    style={'border': 'none'}
                )],
                width=6
            )

        ]),
        style={'margin': '1rem 0'}
    ),

    # ------------ Seção 4 - Storage e máquina geral ------------ #


    dbc.Col(
        dbc.Row([

            dbc.Col([
                dbc.Card([
                    dbc.CardHeader('Utilização de Storage (Cluster + 24x7) por grupo', style={'border-bottom': 'none', 'background-color': 'darkorange', 'color': 'white'}),
                    dcc.Graph(
                        id='graph_storage',
                    )],
                    className='shadow',
                    style={'border': 'none'}
                )],
                width=4
            ),
            
            dbc.Col([
                dbc.Card([
                    dbc.CardHeader('Utilização de Máquina em Cluster total', style={'border-bottom': 'none', 'background-color': 'lightseagreen', 'color': 'white'}),
                    dcc.Graph(
                        id='graph_total',
                    )],
                    className='shadow',
                    style={'border': 'none'}
                )],
                width=8
            )
        ]),
        style={'margin': '1rem 0'}
    )
], style={'background-color':'#212529'}
)

@app.callback(
    Output('fig_storage_card', 'figure'),
    Output('graph_horas_nucleo_total', 'figure'),

    Output('graph_maquina24x7', 'figure'),
    Output('graph_maquina_cluster', 'figure'),
    Output('graph_storage', 'figure'),
    Output('graph_total', 'figure'),

    Output('total_utilizado_storage', 'children'),
    Output('total_disponivel_storage', 'children'),
    Output('total_utilizado_horas_nucleo', 'children'),
    Output('total_disponivel_horas_nucleo', 'children'),
    Output('capacidade_horas_nucleo', 'children'),

    Input('month_slider', 'value')
)

def update_figure(value):
    if value == 12: 
        df_dados = pd.read_excel('relatorios/2021/12-dez.xlsx')
        mes = 31
    elif value == 11: 
        df_dados = pd.read_excel('relatorios/2021/11-nov.xlsx')
        mes = 30
    elif value == 10: 
        df_dados = pd.read_excel('relatorios/2021/10-out.xlsx')
        mes = 31
    elif value == 9: 
        df_dados = pd.read_excel('relatorios/2021/9-set.xlsx')
        mes = 30
    elif value == 8: 
        df_dados = pd.read_excel('relatorios/2021/8-ago.xlsx')
        mes = 31
    elif value == 7: 
        df_dados = pd.read_excel('relatorios/2021/7-jul.xlsx')
        mes = 31
    elif value == 6: 
        df_dados = pd.read_excel('relatorios/2021/6-jun.xlsx')
        mes = 30
    elif value == 5: 
        df_dados = pd.read_excel('relatorios/2021/5-mai.xlsx')
        mes = 31
    elif value == 4: 
        df_dados = pd.read_excel('relatorios/2021/4-abr.xlsx')
        mes = 30
    elif value == 3: 
        df_dados = pd.read_excel('relatorios/2021/3-mar.xlsx')
        mes = 31
    elif value == 2: 
        df_dados = pd.read_excel('relatorios/2021/2-fev.xlsx')
        mes = 28
    else: 
        df_dados = pd.read_excel('relatorios/2021/1-jan.xlsx')
        mes = 31

    # ------------ Máquina em 24x7 -------------- #
        
    df_maquina24x7 = df_dados[['Projeto', 'Máquina em 24x7']]                         
    df_maquina24x7 = df_maquina24x7.dropna()                                                 
    df_maquina24x7 = df_maquina24x7.sort_values(by=['Máquina em 24x7'], ascending=False)

    total_utilizado_24x7 = df_maquina24x7['Máquina em 24x7'].sum()

    fig_maquina24x7 = px.bar(df_maquina24x7.head(10), x="Projeto", y="Máquina em 24x7", color="Projeto")
    fig_maquina24x7.update(layout_showlegend=False)

    # ------------ Máquina em Cluster ------------ #

    df_maquinaCluster = df_dados[['Projeto', 'Máquina em Cluster']]                         
    df_maquinaCluster = df_maquinaCluster.dropna()                                                 
    df_maquinaCluster = df_maquinaCluster.sort_values(by=['Máquina em Cluster'], ascending=False)

    total_utilizado_cluster = df_maquinaCluster['Máquina em Cluster'].sum()

    fig_maquinaCluster = px.bar(df_maquinaCluster, x="Projeto", y="Máquina em Cluster", color="Projeto")
    fig_maquinaCluster.update(layout_showlegend=False)


    # ------------ Horas nucleo total -------------- #

    total_utilizado_horas_nucleo = total_utilizado_24x7 + total_utilizado_cluster
    total_disponivel_horas_nucleo = (2108*24*mes) - total_utilizado_horas_nucleo
    capacidade_horas_nucleo = (2108*24*mes)

    utilizado_horas_nucleo = (total_utilizado_horas_nucleo / capacidade_horas_nucleo) * 100
    utilizado_horas_nucleo = round(utilizado_horas_nucleo, 2)

 
    fig_horas_nucleo_total = go.Figure(
        data=[
            go.Bar(name='24x7', x=[total_utilizado_24x7], y=['Total'], orientation='h', marker_color='rgb(20, 200, 255)'),
            go.Bar(name='Cluster', x=[total_utilizado_cluster], y=['Total'], orientation='h', marker_color='rgb(30, 110, 255)'),
            go.Bar(name='Disponível', x=[total_disponivel_horas_nucleo], y=['Total'], orientation='h', marker_color='#efefef')
        ]
    )

    fig_horas_nucleo_total.update_layout(barmode='stack', yaxis={'visible': False, 'showticklabels': False}, xaxis={'visible': False, 'showticklabels': False, 'showline': False}, autosize=False,
    height=100, plot_bgcolor='rgba(0,0,0,0)', margin=dict(
        l=10,
        r=0,
        b=10,
        t=0
    ))

    fig_horas_nucleo_total.add_annotation(x=0, y=['Total'],
            text="Utilizado", xanchor="left",
            showarrow=False)

    fig_horas_nucleo_total.add_annotation(x=total_utilizado_24x7+total_utilizado_cluster, y=['Total'],
            text=str(utilizado_horas_nucleo)+'%', xanchor="auto",
            showarrow=False)

    fig_horas_nucleo_total.update_layout(legend=dict(
        yanchor="top",
        y=0.7,
        xanchor="right",
        x=1.2
    ))

    # ------------ Storage ----------------------- #

    df_storage = df_dados[['Projeto', 'Storage em cluster(GB)', 'Storage em 24x7(GB)']]                      
    df_storage= df_storage.dropna(thresh=2)                                                        
    df_storage = df_storage.fillna(0)                                                               
    df_storage['Total'] = df_storage['Storage em cluster(GB)'] + df_storage['Storage em 24x7(GB)']    
    df_storage.loc[-1] = ['Disponível', '', '', 134206 - df_storage['Total'].sum()]                 
    df_storage.index = df_storage.index + 1
    df_storage = df_storage.sort_index()

    total_utilizado_storage = df_storage['Total'].sum() - df_storage.loc[0, 'Total']
    total_disponivel_storage = 134206 - total_utilizado_storage                        

    #-- Gráfico --#

    labels = df_storage['Projeto']
    values = df_storage['Total']

    fig_storage = go.Figure(data=[go.Pie(labels = labels, values = values, pull = [0.1])])
    fig_storage.update_traces(textposition='inside', textinfo = 'label+percent')
    fig_storage.update(layout_showlegend=False)



    fig_storage_card = go.Figure(
        data=[
            go.Bar(name='Utilizado', x=[total_utilizado_storage], y=['Total'], orientation='h', marker_color='darkorange'),
            go.Bar(name='Disponível', x=[total_disponivel_storage], y=['Total'], orientation='h', marker_color='#efefef')
        ]
    )

    fig_storage_card.update_layout(barmode='stack', yaxis={'visible': False, 'showticklabels': False}, xaxis={'visible': False, 'showticklabels': False, 'showline': False}, autosize=False,
    height=100, plot_bgcolor='rgba(0,0,0,0)', margin=dict(
        l=10,
        r=0,
        b=10,
        t=0
    ))

    fig_storage_card.add_annotation(x=0, y=['Total'],
            text="Utilizado", xanchor="left",
            showarrow=False)

    utilizado_storage = (total_utilizado_storage / 134206) * 100
    utilizado_storage = round(utilizado_storage, 2)

    fig_storage_card.add_annotation(x=total_utilizado_storage, y=['Total'],
            text=str(utilizado_storage)+'%', xanchor="auto",
            showarrow=False)

    fig_storage_card.update_layout(legend=dict(
        yanchor="top",
        y=0.5,
        xanchor="right",
        x=1.2
    ))

    # ------------ total ------------------------- #

    df_relatorio_total = pd.DataFrame()
    #cont = 0
    for dataframe in os.listdir('relatorios/2021'):
        df_relatorio_total = df_relatorio_total.append(pd.read_excel(os.path.join('relatorios/2021', dataframe)))    

    df_total = df_relatorio_total[['Projeto', 'Máquina em Cluster', 'Mês']]             
    df_total = df_total.dropna()
    df_total = df_total.sort_values(by=['Mês'])                                                          

    fig_total = px.line(df_total, 
                        x = 'Mês', 
                        y = 'Máquina em Cluster',
                        color = 'Projeto'
    )

    return [
            fig_storage_card,
            fig_horas_nucleo_total,

            fig_maquina24x7, 
            fig_maquinaCluster, 
            fig_storage,
            fig_total,

            total_utilizado_storage, 
            total_disponivel_storage,
            total_utilizado_horas_nucleo, 
            total_disponivel_horas_nucleo,
            capacidade_horas_nucleo
           ]

app.run_server(mode='external')
# app.run_server(mode='inline')

/Users/tiagoferreto/opt/miniconda3/lib/python3.8/site-packages/jupyter_dash/jupyter_app.py:139: UserWarning:

The 'environ['werkzeug.server.shutdown']' function is deprecated and will be removed in Werkzeug 2.1.



Dash app running on http://127.0.0.1:8050/
